In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
load_dotenv("./../config/credentials_my.env")

print(f'Project Connection String: <{os.environ["PROJECT_CONNECTION_STRING"][:30]}>')

Project Connection String: <westus.api.azureml.ms;eca2eddb>


In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai01-rg',
 'project_name': 'mmai01-hub-project01-vhdt'}

In [3]:
from azure.ai.projects.models import CodeInterpreterTool

code_interpreter = CodeInterpreterTool()

print(f"Code interpreter definitions: {code_interpreter.definitions}")
print(f"Code interpreter resources: {code_interpreter.resources}")
print(f"Code interpreter file_ids: {code_interpreter.file_ids}")

Code interpreter definitions: [{'type': 'code_interpreter'}]
Code interpreter resources: {}
Code interpreter file_ids: set()


In [5]:
agent = project_client.agents.create_agent(
    model="gpt-4o-mini", # gpt-4o-mini or gpt-4o
    name="my-agent",
    instructions="You are a helpful assistant",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources        
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_9zKkNvFXw5XfiYx5IN2bIixD', 'object': 'assistant', 'created_at': 1735495827, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}>

project_client.agents.delete_agent(agent_02.id)

In [6]:
#upload a file
from azure.ai.projects.models import FilePurpose
file = project_client.agents.upload_file(file_path='./README.md', purpose="assistants") # or FilePurpose.AGENTS
print(f"Uploaded file, file ID: {file.id}")

HttpResponseError: (None) The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at oai-assistants@microsoft.com if you keep seeing this error. (Please include the request ID 8fb347deb891656d6b81093d9ca1f0ee in your email.)
Code: None
Message: The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at oai-assistants@microsoft.com if you keep seeing this error. (Please include the request ID 8fb347deb891656d6b81093d9ca1f0ee in your email.)

In [ ]:
# Delete all agents
for pca in project_client.agents.list_agents()['data']:
    print(f"Deleting agent {pca.name} ({pca.id})...")
    project_client.agents.delete_agent(pca.id)